In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import json
import tensorflow as tf
from models import Rnn
from datasets import ZeroShotDataset
from algomorphism.methods.nn import pca_denoising_preprocessing
from algomorphism.figures.nn import pca_denoising_figure, print_confmtx
from algomorphism.figures.nn import multiple_models_history_figure
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# optional for GPU usage

# for gpu in tf.config.list_physical_devices('GPU'):
#     print(gpu)
#     tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
embs_id = '64'
embsD = 64

dz = ZeroShotDataset(embs_id)
labels = ['0','1','2']
# all combinations seen (s), unseen (u):
# {s: ['0','1'], u:['2']}, {s: ['0','2'], u:['1']}, {s: ['1','2'], u:['0']}
seen_labels = ['0','2']
unseen_labels = ['1']
outfilekey = 's-{}-u-{}'.format('-'.join(seen_labels), '-'.join(unseen_labels))

dz.load_data(seen_labels,unseen_labels)

knn = KNeighborsClassifier(1)
r_emb = dz.r_emb_disct_preprosesing(seen_labels, unseen_labels)
X = tf.concat([r_emb[label].reshape(1,-1) for label in r_emb.keys()],axis=0)
Xall = tf.concat([r_emb[key].reshape(1,-1) for key in r_emb.keys()],axis=0)
labels_all = list(r_emb.keys())
Y = dz.get_r_onehot_from_labels([label for label in r_emb.keys()])
knn.fit(X,Y)

In [ ]:
for x, y_emb ,y, y1 in dz.val.unseen:
    x = x.numpy()[np.where(
        np.logical_and(
            (y.numpy()==[1,0,0]).sum(axis=1)==3,
            (y1.numpy()==[0,1]).sum(axis=1)==2
        ))]
    for i in range(x.shape[0]):
        plt.figure(figsize=(8,2.5))
        plt.plot(x[i],'-o')
        plt.xlabel(r'$time(s)$')
        plt.ylabel(r'$Normalized\; UDP\; Throughput$')
        plt.show()
        break
    break

In [ ]:
es = {'es_strategy':'patience', 'es_metric':'val_unseen_cost', 'es_patience': 10, 'es_min_delta': 1e-4}

es = {'es_strategy':'first_drop', 'es_metric':'val_harmonic'}
# add outfilekey as weights_outfile
rnn = Rnn(dz, embsD, knn, es, weights_outfile=['.', outfilekey],
          optimizer="Adam", learning_rate=1e-4
        )

In [ ]:
rnn.train(dz, 200, ['train', 'val'])
# rnn.load_weights("{}/weights/weights_best_{}.tf".format('.', outfilekey))

In [ ]:
save_history_to_file = False
if save_history_to_file:
    with open('./data/2ornn/history-{}.json'.format(outfilekey), 'w') as outfile:
            json.dump(rnn.history, outfile, indent=4)
else:
    with open('./data/2ornn/history-{}.json'.format(outfilekey), 'r') as f:
        history = json.load(f)

In [ ]:
save_obj = ['.', '.', '2ornn', 'cost-score', outfilekey]
multiple_models_history_figure([rnn], save_obj=None)

In [ ]:
val_seen = rnn.score_mtr.metric_dataset(dz.val.seen)
val_unseen = rnn.score_mtr.metric_dataset(dz.val.unseen)

print('val seen', val_seen)
print('val unseen', val_unseen)
print('HM', 2*val_seen*val_unseen/(val_seen + val_unseen))


print_confmtx(rnn, dz, outfilekey+'_r_', -2, -2)
print_confmtx(rnn, dz, outfilekey+'_a_', -1, -1)

In [ ]:
pca_dict, pca_emb, knn_pca= pca_denoising_preprocessing(rnn, dz, X, Y, pca_emb_idxs=[0, 1, 2], example_predicted_types=['val_seen', 'val_unseen'])
save_obj = ['.','.','2ornn', 'node-emb', outfilekey]
pca_denoising_figure(pca_dict, pca_emb, knn_pca, r_emb.keys(), save_obj=None, pca_emb_idxs=[0,1,2])

In [ ]:
# p_train = p_val = p_test = None
# for x, _, _, _ in dz.train:
#     p_train = rnn.call(x)
# for x, _, _, _ in dz.val:
#     p_val = rnn.call(x)
# for x, _, _, _ in dz.test:
#     p_test = rnn.call(x)
#
# concat = np.concatenate([Xall, p_train[0], p_val[0], p_test[0]])
# mylist = labels_all+['train']*p_train[0].shape[0]+['val']*p_val[0].shape[0]+['test']*p_test[0].shape[0]
#
# ediml = np.array([
#     (i, l) for i, l in enumerate(mylist)
# ])
# out = 'foo.tsv'
# with open("./tensorboard_files/{}".format(out), "w") as foo:
#     text = "idx\tlabel\n"
#     for c in ediml:
#         text += '\t'.join(c.tolist())
#         text += '\n'
#     foo.write(text)
#     foo.close()
#

In [ ]:
# %load_ext tensorboard
# %reload_ext tensorboard
#
# log_dir="./tensorboard_files/"
# if not os.path.exists(log_dir):
#     os.makedirs(log_dir)
# config = projector.ProjectorConfig()
# embedding = config.embeddings.add()
#
# weights = tf.Variable(concat)
# # Create a checkpoint from embedding, the filename and key are
# # name of the tensor.
# checkpoint = tf.train.Checkpoint(embedding=weights)
# checkpoint.save(os.path.join(log_dir, "embedding"))
#
#
# # The name of the tensor will be suffixed by `/.ATTRIBUTES/VARIABLE_VALUE`
# embedding.tensor_name = "embedding/.ATTRIBUTES/VARIABLE_VALUE"
# # set tsv file
# embedding.metadata_path = out
# projector.visualize_embeddings(log_dir, config)
#
# # ref: from tensorflow site
#

In [ ]:
# %tensorboard --logdir=./tensorboard_files/ --host localhost --port 8882